In [ ]:
import numpy as np
import plotly.graph_objects as go
from math import pi

np.random.seed(42)

fs = 4000  # Hz
Ts = 1 / fs
time = np.arange(0, 20e-3, Ts)  # seconds

# Generate periodic signals
f1 = 50
T1 = 1 / f1
fmax = 20 * f1
F = np.arange(0, fmax, f1)
P = np.random.uniform(-pi, pi, len(F))
A = np.random.uniform(0, 0.1, len(F))
A[1] = 1
periodic = np.zeros((len(F),len(time)))
for i in range(len(F)):
    for j, t in enumerate(time):
        periodic[i,j] = A[i] * np.cos(2 * pi * F[i] * t + P[i])
periodics = np.sum(periodic, axis=0)
# print("Periodic components")
# print("Frequencies (Hz): Amplitudes")
# for i in range(len(F)):
#     print(f"{F[i]}: {A[i]}")

# Generate white noise
noise_max = 0.1
noise = np.random.uniform(0, noise_max, len(time))
print(f"Maximum noise amplitude = {noise_max}")

# Generate aperiodic component
fade_coef = 100
amp_coef = 1
t_start = time[len(time) // 2]  # middle time
aperiodic = amp_coef * np.exp(-fade_coef * (time - t_start))
aperiodic[time < t_start] = 0  # aperiodic starts form t_start

# Sum up all components into one signal
signal = 0
signal += periodics
# signal += aperiodic
# signal += noise

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=signal, name="signal", mode="lines+markers", line=dict(color="black")))
fig.add_trace(go.Scatter(x=time, y=periodic[1], name="50 Hz", mode="lines+markers", line=dict(color="red")))
# fig.add_trace(go.Scatter(y=periodic[0], name="DC"))
# fig.add_trace(go.Scatter(y=np.sum(periodic[2:], axis=0), name="harmonics"))
# fig.add_trace(go.Scatter(y=aperiodic, name="aperiodic"))
# fig.add_trace(go.Scatter(y=noise, name="noise"))
fig.update_layout(title="Digital Signal", xaxis_title="Time, s", yaxis_title="Magnitude, linear")
fig.show()

N = len(signal)
res = np.fft.fft(signal,N) / N # complex
freqs = np.fft.fftfreq(len(signal),Ts)
res = np.fft.fftshift(res)[N//2:] * 2
freqs = np.fft.fftshift(freqs)[N//2:]

mags = np.abs(res)

fig = go.Figure()
# Add the stems (vertical lines)
for i in range(len(freqs)):
    fig.add_trace(
        go.Scatter(
            x=[freqs[i], freqs[i]],
            y=[0, mags[i]],
            mode="lines",
            line=dict(color="black", width=2),
            showlegend=False,
        )
    )
# Add the markers at stems
fig.add_trace(
    go.Scatter(
        x=freqs,
        y=mags,
        mode="markers",
        marker=dict(color="black", size=8),
        name="Signal",
    )
)
fig.add_trace(
    go.Scatter(
        x=[freqs[1], freqs[1]],
        y=[0, mags[1]],
        mode="lines",
        line=dict(color="red", width=1),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter(
        x=[freqs[1]],
        y=[mags[1]],
        mode="markers",
        marker=dict(color="red", size=4),
        name="Filtered signal",
    )
)
fig.update_layout(title="DFT Spectrum", xaxis_title="Frequency, Hz", yaxis_title="Magnitude, linear")
fig.show()


Maximum noise amplitude = 0.1


In [36]:
freqs

array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,
        450.,  500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,
        900.,  950., 1000., 1050., 1100., 1150., 1200., 1250., 1300.,
       1350., 1400., 1450., 1500., 1550., 1600., 1650., 1700., 1750.,
       1800., 1850., 1900., 1950.])

In [37]:
N

80